In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import bisect
from scipy.stats import norm

In [2]:
# Load the data file
file_path = './Options_SPX.csv'
data = pd.read_csv(file_path)

# Black-Scholes call option pricing formula
# Removed Crank-Nicholson and replaced with Binomial Tree method for pricing

def binomial_tree_call_price(S, K, T, r, sigma, N=100):
    dt = T / N  # Time step size
    u = np.exp(sigma * np.sqrt(dt))  # Up factor
    d = 1 / u  # Down factor
    p = (np.exp(r * dt) - d) / (u - d)  # Risk-neutral probability

    # Initialize asset prices at maturity
    asset_prices = np.zeros(N + 1)
    for i in range(N + 1):
        asset_prices[i] = S * (u ** (N - i)) * (d ** i)

    # Initialize option values at maturity
    option_values = np.maximum(asset_prices - K, 0)

    # Backward induction to calculate option value at t=0
    for j in range(N - 1, -1, -1):
        for i in range(j + 1):
            option_values[i] = np.exp(-r * dt) * (p * option_values[i] + (1 - p) * option_values[i + 1])

    return option_values[0]

In [3]:
# Function to calculate implied volatility
def implied_volatility(C_mkt, S, K, T, r):
    def objective(sigma):
        return binomial_tree_call_price(S, K, T, r, sigma) - C_mkt
    try:
        return bisect(objective, 1e-6, 5)  # Searching for sigma in a reasonable range
    except ValueError:
        return np.nan

# Initialize strategy variables
position = 0  # Current asset position
cash = 0      # Cash balance
portfolio_values = []  # Store portfolio values over time
predicted_prices = []  # Store predicted option prices

# Use a dynamic window of 5 rows for prediction and hedging
window_size = 5
for start in range(0, len(data), window_size):
    window_data = data[start:start + window_size]
    if len(window_data) < window_size:
        break  # Skip incomplete windows

    for i, row in window_data.iterrows():
        S = row['S']  # Current stock price
        K = S * np.exp(-row['Moneyness'] / 100)  # Calculate strike price based on Moneyness
        T = row['TTM'] / 252  # Convert time to expiration to years
        r = row['R'] / 100  # Convert interest rate to decimal
        C_mkt = row['C_mkt']  # Market option price

        # Calculate implied volatility
        sigma = implied_volatility(C_mkt, S, K, T, r)

        # Skip this row if implied volatility could not be calculated
        if np.isnan(sigma):
            continue

        # Calculate Delta using Binomial Tree method
        epsilon = 1e-4  # Small change for Delta calculation
        V_up = binomial_tree_call_price(S * (1 + epsilon), K, T, r, sigma)
        V_down = binomial_tree_call_price(S * (1 - epsilon), K, T, r, sigma)
        delta = (V_up - V_down) / (2 * S * epsilon)

        # Calculate predicted option price using Binomial Tree method
        predicted_price = binomial_tree_call_price(S, K, T, r, sigma)
        predicted_prices.append(predicted_price)

        # Determine the target position based on Delta
        target_position = -delta
        position_change = target_position - position

        # Update cash and asset position
        cash -= position_change * S
        position = target_position

        # Record portfolio value
        portfolio_values.append(position * S + cash)

# Convert portfolio values to a DataFrame for further analysis
portfolio_values = pd.DataFrame(portfolio_values, columns=['Portfolio Value'])

# Print predicted prices
predicted_prices_df = pd.DataFrame(predicted_prices, columns=['Predicted Price'])
# print(predicted_prices_df.head())
# print(data.head())

print(((data['C_mkt'] - predicted_prices)**2).mean())

# Calculate risk metrics
portfolio_values['Returns'] = portfolio_values['Portfolio Value'].pct_change().dropna()
volatility = portfolio_values['Returns'].std()  # Calculate return volatility
cumulative_returns = (1 + portfolio_values['Returns']).cumprod()
drawdown = cumulative_returns.cummax() - cumulative_returns  # Calculate drawdown
max_drawdown = drawdown.max()

# Output risk metrics
volatility, max_drawdown

/var/folders/bn/0_n9dq9s6m51l0sffgs_yvtw0000gn/T/ipykernel_7301/485742796.py:25: RuntimeWarning: overflow encountered in scalar multiply
  option_values[i] = np.exp(-r * dt) * (p * option_values[i] + (1 - p) * option_values[i + 1])
/var/folders/bn/0_n9dq9s6m51l0sffgs_yvtw0000gn/T/ipykernel_7301/485742796.py:25: RuntimeWarning: invalid value encountered in scalar add
  option_values[i] = np.exp(-r * dt) * (p * option_values[i] + (1 - p) * option_values[i + 1])
/var/folders/bn/0_n9dq9s6m51l0sffgs_yvtw0000gn/T/ipykernel_7301/485742796.py:25: RuntimeWarning: overflow encountered in scalar add
  option_values[i] = np.exp(-r * dt) * (p * option_values[i] + (1 - p) * option_values[i + 1])


ValueError: operands could not be broadcast together with shapes (1008,) (876,) 